# Installation 

In [46]:
import pandas as pd
import numpy as np
import requests
import time
import neo4j
import json

In [47]:
API_KEY = '39Lmr8g-EvTQ_EuO1ZnoNtri4K5NXh9cBB4zUl6vpV0' 

# Merging Datasets

In [48]:
zhvi = pd.read_csv('zhvi_county.csv')
# print(zhvi)

In [49]:
variables_definations_for_B17017 = {
    "B17017_002E":  "Income in the past 12 months below poverty level",
    "B17017_031E": "Income in the past 12 months at or above poverty level"
}

index_for_variables_B17017 = {
    "B17017_002E": 4,
    "B17017_031E": 120,
    "GEO_ID" : 236,
    "NAME" : 237
}

In [50]:
B17017_json = json.load(open('B17017_Data.json'))

In [51]:
i = 0
for var in B17017_json[0]:
    print(f'{i} : {var}')
    i += 1 

0 : B17017_001E
1 : B17017_001EA
2 : B17017_001M
3 : B17017_001MA
4 : B17017_002E
5 : B17017_002EA
6 : B17017_002M
7 : B17017_002MA
8 : B17017_003E
9 : B17017_003EA
10 : B17017_003M
11 : B17017_003MA
12 : B17017_004E
13 : B17017_004EA
14 : B17017_004M
15 : B17017_004MA
16 : B17017_005E
17 : B17017_005EA
18 : B17017_005M
19 : B17017_005MA
20 : B17017_006E
21 : B17017_006EA
22 : B17017_006M
23 : B17017_006MA
24 : B17017_007E
25 : B17017_007EA
26 : B17017_007M
27 : B17017_007MA
28 : B17017_008E
29 : B17017_008EA
30 : B17017_008M
31 : B17017_008MA
32 : B17017_009E
33 : B17017_009EA
34 : B17017_009M
35 : B17017_009MA
36 : B17017_010E
37 : B17017_010EA
38 : B17017_010M
39 : B17017_010MA
40 : B17017_011E
41 : B17017_011EA
42 : B17017_011M
43 : B17017_011MA
44 : B17017_012E
45 : B17017_012EA
46 : B17017_012M
47 : B17017_012MA
48 : B17017_013E
49 : B17017_013EA
50 : B17017_013M
51 : B17017_013MA
52 : B17017_014E
53 : B17017_014EA
54 : B17017_014M
55 : B17017_014MA
56 : B17017_015E
57 : B17017_0

In [52]:
B17017_data = []
for var in B17017_json:
    B17017_data.append([var[index_for_variables_B17017['GEO_ID']], var[index_for_variables_B17017['NAME']], var[index_for_variables_B17017['B17017_002E']], var[index_for_variables_B17017['B17017_031E']]])

In [95]:
B17017_data

[['GEO_ID', 'NAME', 'B17017_002E', 'B17017_031E'],
 ['0500000US01003', 'Baldwin County, Alabama', '14013', '84841'],
 ['0500000US01015', 'Calhoun County, Alabama', '8121', '37580'],
 ['0500000US01043', 'Cullman County, Alabama', '6423', '29543'],
 ['0500000US01049', 'DeKalb County, Alabama', '5801', '20658'],
 ['0500000US01051', 'Elmore County, Alabama', '3869', '30192'],
 ['0500000US01055', 'Etowah County, Alabama', '6917', '33039'],
 ['0500000US01069', 'Houston County, Alabama', '7136', '35281'],
 ['0500000US01073', 'Jefferson County, Alabama', '43797', '228080'],
 ['0500000US01077', 'Lauderdale County, Alabama', '5110', '33911'],
 ['0500000US01081', 'Lee County, Alabama', '13009', '58821'],
 ['0500000US01083', 'Limestone County, Alabama', None, None],
 ['0500000US01089', 'Madison County, Alabama', '19238', '147216'],
 ['0500000US01095', 'Marshall County, Alabama', '6410', '30201'],
 ['0500000US01097', 'Mobile County, Alabama', '31152', '131453'],
 ['0500000US01101', 'Montgomery Coun

In [53]:
variables_definations_for_S2502 = {
    "S2502_C01_001EA":  "Total Occupied Housing Units",
    "S2502_C01_008E": "two or more races",
    "S2502_C01_009E" : "Hispanic or Latino origin",
    "S2502_C01_010E" : "White alone, not Hispanic or Latino",
    "S2502_C01_011E" : "under 35 years old",
    "S2502_C01_012E" : "35 to 44 years old",
    "S2502_C01_013E" : "45 to 54 years old",
    "S2502_C01_014E" : "55 to 64 years old",
    "S2502_C01_015E" : "65 to 74 years old",
    "S2502_C01_016E" : "75 to 84 years old",
    "S2502_C01_017E" : "85 years and over",
    "S2502_C01_018E" : "Less than high school graduate",
    "S2502_C01_019E" : "High school graduate (includes equivalency)",
    "S2502_C01_020E" : "Some college or associate's degree",
    "S2502_C01_021E" : "Bachelor's degree or higher",
}

index_for_variables_S2502 = {
    "S2502_C01_001EA": 2,
    "S2502_C01_008E": 30,
    "S2502_C01_009E" : 34,
    "S2502_C01_010E" : 38,
    "S2502_C01_011E": 42,
    "S2502_C01_012E" : 46,
    "S2502_C01_013E": 50,
    "S2502_C01_014E" : 54,
    "S2502_C01_015E" : 58,
    "S2502_C01_016E" : 62,
    "S2502_C01_017E" : 66,
    "S2502_C01_018E" : 70,
    "S2502_C01_019E" : 74,
    "S2502_C01_020E" : 78,
    "S2502_C01_021E" : 82,
    "GEO_ID" : 0,
    "NAME" : 1
}

In [54]:
S2502_json = json.load(open('S2502_Data.json'))

In [55]:
i = 0
for var in S2502_json[0]:
    print(f'{i} : {var} : {S2502_json[1][i]}')
    i += 1 

0 : GEO_ID : 0500000US01003
1 : NAME : Baldwin County, Alabama
2 : S2502_C01_001E : 98854
3 : S2502_C01_001EA : None
4 : S2502_C01_001M : 3781
5 : S2502_C01_001MA : None
6 : S2502_C01_002E : 84222
7 : S2502_C01_002EA : None
8 : S2502_C01_002M : 3109
9 : S2502_C01_002MA : None
10 : S2502_C01_003E : 8400
11 : S2502_C01_003EA : None
12 : S2502_C01_003M : 1596
13 : S2502_C01_003MA : None
14 : S2502_C01_004E : 273
15 : S2502_C01_004EA : None
16 : S2502_C01_004M : 175
17 : S2502_C01_004MA : None
18 : S2502_C01_005E : 442
19 : S2502_C01_005EA : None
20 : S2502_C01_005M : 270
21 : S2502_C01_005MA : None
22 : S2502_C01_006E : 71
23 : S2502_C01_006EA : None
24 : S2502_C01_006M : 131
25 : S2502_C01_006MA : None
26 : S2502_C01_007E : 1494
27 : S2502_C01_007EA : None
28 : S2502_C01_007M : 789
29 : S2502_C01_007MA : None
30 : S2502_C01_008E : 3952
31 : S2502_C01_008EA : None
32 : S2502_C01_008M : 1104
33 : S2502_C01_008MA : None
34 : S2502_C01_009E : 3552
35 : S2502_C01_009EA : None
36 : S2502_C01_0

In [56]:
S2502_data = []
for var in S2502_json:
    S2502_data.append([var[index_for_variables_S2502['GEO_ID']],
                     var[index_for_variables_S2502['NAME']],
                       var[index_for_variables_S2502['S2502_C01_001EA']],
                         var[index_for_variables_S2502['S2502_C01_008E']],
                           var[index_for_variables_S2502['S2502_C01_009E']],
                             var[index_for_variables_S2502['S2502_C01_010E']],
                               var[index_for_variables_S2502['S2502_C01_011E']],
                                 var[index_for_variables_S2502['S2502_C01_012E']],
                                   var[index_for_variables_S2502['S2502_C01_013E']],
                                     var[index_for_variables_S2502['S2502_C01_014E']],
                                       var[index_for_variables_S2502['S2502_C01_015E']],
                                         var[index_for_variables_S2502['S2502_C01_016E']],
                                           var[index_for_variables_S2502['S2502_C01_017E']],
                                             var[index_for_variables_S2502['S2502_C01_018E']],
                                               var[index_for_variables_S2502['S2502_C01_019E']],
                                                 var[index_for_variables_S2502['S2502_C01_020E']],
                                                   var[index_for_variables_S2502['S2502_C01_021E']],
                                                    int(var[index_for_variables_S2502['S2502_C01_012E']])+int(var[ index_for_variables_S2502['S2502_C01_013E']])+int(var[index_for_variables_S2502['S2502_C01_014E']])+int(var[index_for_variables_S2502['S2502_C01_015E']])+int(var[index_for_variables_S2502['S2502_C01_016E']])+int(var[index_for_variables_S2502['S2502_C01_017E']]) if (var[0] != "GEO_ID") else "Above 35 years old"] )

In [94]:
import pandas as pd

# Define the column names for the DataFrame
column_names = [
    'GEO_ID', 'NAME', 'Total Occupied Housing Units', 'two or more races',
    'Hispanic or Latino origin', 'White alone, not Hispanic or Latino',
    'under 35 years old', '35 to 44 years old', '45 to 54 years old',
    '55 to 64 years old', '65 to 74 years old', '75 to 84 years old',
    '85 years and over', 'Less than high school graduate',
    'High school graduate (includes equivalency)',
    "Some college or associate's degree", "Bachelor's degree or higher",
    'Above 35 years old'
]

# Create the DataFrame
df_S2502 = pd.DataFrame(S2502_data[1:], columns=column_names)  # Skip header row

# Display the DataFrame
print(df_S2502)


             GEO_ID                                  NAME  \
0    0500000US01003               Baldwin County, Alabama   
1    0500000US01015               Calhoun County, Alabama   
2    0500000US01043               Cullman County, Alabama   
3    0500000US01049                DeKalb County, Alabama   
4    0500000US01051                Elmore County, Alabama   
..              ...                                   ...   
843  0500000US72113          Ponce Municipio, Puerto Rico   
844  0500000US72127       San Juan Municipio, Puerto Rico   
845  0500000US72135       Toa Alta Municipio, Puerto Rico   
846  0500000US72137       Toa Baja Municipio, Puerto Rico   
847  0500000US72139  Trujillo Alto Municipio, Puerto Rico   

    Total Occupied Housing Units two or more races Hispanic or Latino origin  \
0                          98854              3952                      3552   
1                          45701               998                      1190   
2                          

In [57]:
variables_definations_for_S2503 = {
    "S2503_C01_026E" : "Less than $20,000,Less than 20 percent",
    "S2503_C01_028E" : "Less than $20,000,30 percent or more",
    "S2503_C01_030E" : "$20,000 to $34,999,Less than 20 percent",
    "S2503_C01_032E" : "$20,000 to $34,999,30 percent or more",
    "S2503_C01_034E" : "$35,000 to $49,999,Less than 20 percent",
    "S2503_C01_036E" : "$35,000 to $49,999,30 percent or more",
    "S2503_C01_038E" : "$50,000 to $74,999,Less than 20 percent",
    "S2503_C01_040E" : "$50,000 to $74,999,30 percent or more",
    "S2503_C01_042E" :  "$75,000 or more!!Less than 20 percent",
    "S2503_C01_044E" :  "$75,000 or more!!30 percent or more"
}

index_for_variables_S2503 = {
    "S2503_C01_026E" : 102,
    "S2503_C01_028E" : 110,
    "S2503_C01_030E" : 118,
    "S2503_C01_032E" : 126,
    "S2503_C01_034E" : 134,
    "S2503_C01_036E" : 142,
    "S2503_C01_038E" : 150,
    "S2503_C01_040E" : 158,
    "S2503_C01_042E" : 166,
    "S2503_C01_044E" : 174,
    "GEO_ID" : 0,
    "NAME" : 1
}

In [58]:
S2503_json = json.load(open('S2503_Data.json'))

In [59]:
i = 0
for var in S2503_json[0]:
    print(f'{i} : {var} : {S2503_json[1][i]}')
    i += 1 

0 : GEO_ID : 0500000US01003
1 : NAME : Baldwin County, Alabama
2 : S2503_C01_001E : 98854
3 : S2503_C01_001EA : None
4 : S2503_C01_001M : 3781
5 : S2503_C01_001MA : None
6 : S2503_C01_002E : 4555
7 : S2503_C01_002EA : None
8 : S2503_C01_002M : 1777
9 : S2503_C01_002MA : None
10 : S2503_C01_003E : 945
11 : S2503_C01_003EA : None
12 : S2503_C01_003M : 567
13 : S2503_C01_003MA : None
14 : S2503_C01_004E : 3760
15 : S2503_C01_004EA : None
16 : S2503_C01_004M : 1562
17 : S2503_C01_004MA : None
18 : S2503_C01_005E : 4962
19 : S2503_C01_005EA : None
20 : S2503_C01_005M : 1542
21 : S2503_C01_005MA : None
22 : S2503_C01_006E : 4615
23 : S2503_C01_006EA : None
24 : S2503_C01_006M : 1547
25 : S2503_C01_006MA : None
26 : S2503_C01_007E : 6643
27 : S2503_C01_007EA : None
28 : S2503_C01_007M : 1940
29 : S2503_C01_007MA : None
30 : S2503_C01_008E : 10949
31 : S2503_C01_008EA : None
32 : S2503_C01_008M : 2363
33 : S2503_C01_008MA : None
34 : S2503_C01_009E : 15916
35 : S2503_C01_009EA : None
36 : S250

In [60]:
S2503_Data = []
for var in S2503_json:
    S2503_Data.append([var[index_for_variables_S2503['GEO_ID']],
                     var[index_for_variables_S2503['NAME']],
                        var[index_for_variables_S2503['S2503_C01_026E']],
                        var[index_for_variables_S2503['S2503_C01_028E']],
                        var[index_for_variables_S2503['S2503_C01_030E']],
                        var[index_for_variables_S2503['S2503_C01_032E']],
                        var[index_for_variables_S2503['S2503_C01_034E']],
                        var[index_for_variables_S2503['S2503_C01_036E']],
                        var[index_for_variables_S2503['S2503_C01_038E']],
                        var[index_for_variables_S2503['S2503_C01_040E']],
                        var[index_for_variables_S2503['S2503_C01_042E']],
                        var[index_for_variables_S2503['S2503_C01_044E']],
                        int(var[index_for_variables_S2503['S2503_C01_026E']]) + int(var[index_for_variables_S2503['S2503_C01_030E']]) if (var[0] != 'GEO_ID') else "less than 35k income, less 20 percent",
                        int(var[index_for_variables_S2503['S2503_C01_028E']]) + int(var[index_for_variables_S2503['S2503_C01_032E']]) if (var[0] != 'GEO_ID') else "less than 35k income, 30 percent or more",
                        int(var[index_for_variables_S2503['S2503_C01_034E']]) + int(var[index_for_variables_S2503['S2503_C01_038E']]) if (var[0] != 'GEO_ID') else "35k to 75k income, 20 percent or less",
                        int(var[index_for_variables_S2503['S2503_C01_036E']]) + int(var[index_for_variables_S2503['S2503_C01_040E']]) if (var[0] != 'GEO_ID') else "35k to 75k income, 30 percent or more"])
                       

In [93]:
np.shape(S2503_Data)

(849, 16)

In [91]:
import pandas as pd

# Define the column names for the DataFrame
column_names = [
    'GEO_ID', 'NAME', 'Less than $20,000,Less than 20 percent',
    'Less than $20,000,30 percent or more',
    '$20,000 to $34,999,Less than 20 percent',
    '$20,000 to $34,999,30 percent or more',
    '$35,000 to $49,999,Less than 20 percent',
    '$35,000 to $49,999,30 percent or more',
    '$50,000 to $74,999,Less than 20 percent',
    '$50,000 to $74,999,30 percent or more',
    '$75,000 or more!!Less than 20 percent',
    '$75,000 or more!!30 percent or more', 
    'Less than 35k income, less 20 percent',
    'Less than 35k income, 30 percent or more',
    '35k to 75k income, 20 percent or less',
    '35k to 75k income, 30 percent or more'
]

# Create the DataFrame
df_S2503 = pd.DataFrame(S2503_Data[1:], columns=column_names)  # Skip header row

# Display the DataFrame
print(df_S2503)


             GEO_ID                                  NAME  \
0    0500000US01003               Baldwin County, Alabama   
1    0500000US01015               Calhoun County, Alabama   
2    0500000US01043               Cullman County, Alabama   
3    0500000US01049                DeKalb County, Alabama   
4    0500000US01051                Elmore County, Alabama   
..              ...                                   ...   
843  0500000US72113          Ponce Municipio, Puerto Rico   
844  0500000US72127       San Juan Municipio, Puerto Rico   
845  0500000US72135       Toa Alta Municipio, Puerto Rico   
846  0500000US72137       Toa Baja Municipio, Puerto Rico   
847  0500000US72139  Trujillo Alto Municipio, Puerto Rico   

    Less than $20,000,Less than 20 percent  \
0                                     1149   
1                                      558   
2                                      629   
3                                     1019   
4                                    

In [107]:
URI = "neo4j+s://5dd34660.databases.neo4j.io"
AUTH = ("neo4j", "a1yUVTX6sa3eGz7MSVAqED0zzIB_VUq4YAsSPlVB-JU")
def main():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            # Populate Neo4j with county data from original dataset
            # for state in zhvi['StateName'].unique():       
            #     add_counties_for_state(session, state)
            #     print(f"Counties added for State {state}")

            # Populate Neo4j with poverty data
            populate_poverty_data(driver, B17017_data)
            check_poverty_data(session)

            # Populate Neo4j with housing data from S2502 dataset
            populate_housing_data(driver, df_S2502)
            check_housing_data(session, "S2502")

            # Populate Neo4j with housing data from S2503 dataset
            populate_housing_data_S2503(driver, df_S2503)
            check_housing_data(session, "S2503")


def check_poverty_data(session):
    result = session.run("MATCH (c:County) WHERE c.IncomeBelowPoverty IS NOT NULL AND c.IncomeAtOrAbovePoverty IS NOT NULL RETURN COUNT(c) AS count")
    count = result.single()["count"]
    if count > 0:
        print(f"Poverty data populated for {count} counties.")
    else:
        print("No poverty data populated.")



def populate_housing_data_S2503(driver, df_S2503):
    with driver.session(database="neo4j") as session:
        for index, row in df_S2503.iterrows():
            geo_id, name, *housing_data = row

            # Extract county name and state abbreviation
            county_info = name.split(", ")
            county_name = county_info[0]
            state_abbr = county_info[1]

            # Match county with corresponding node in Neo4j
            result = session.run("""
                MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
                RETURN c
                """, state_abbr=state_abbr, county_name=county_name)

            for record in result:
                county_node = record["c"]

                # Update or create housing data properties
                housing_data_dict = {column: value for column, value in zip(df_S2503.columns[2:], housing_data)}
                county_node.update(housing_data_dict)
                print(f"Housing data (S2503) populated for County {county_name}.")


def add_counties_for_state(session, state_name):
    # Fetch county names for the given state
    state_filtered_data = zhvi[zhvi['StateName'] == state_name]
    county_names = state_filtered_data['RegionName'].unique()

    # Create or update state node
    session.write_transaction(create_or_update_state_node, state_name)

    # Create "HAS_COUNTY" relationships for the state
    for county_name in county_names:
        session.write_transaction(create_or_update_county_node, county_name)
        session.write_transaction(create_has_county_relationship, state_name, county_name)


def create_or_update_state_node(tx, state_name):
    # Create or update State node with given name
    tx.run("""
        MERGE (s:State {name: $name})
        RETURN s.name AS name
        """, name=state_name
    )


def create_or_update_county_node(tx, county_name):
    # Create or update County node with given name
    tx.run("""
        MERGE (c:County {name: $name})
        RETURN c.name AS name
        """, name=county_name
    )


def create_has_county_relationship(tx, state_name, county_name):
    # Create a "HAS_COUNTY" relationship between state and county
    tx.run("""
        MATCH (s:State {name: $state_name}), (c:County {name: $county_name})
        MERGE (s)-[:HAS_COUNTY]->(c)
        """, state_name=state_name, county_name=county_name
    )


def populate_poverty_data(driver, B17017_data):
    with driver.session(database="neo4j") as session:
        for row in B17017_data[1:]:  # Skip header row
            geo_id, name, below_poverty, at_or_above_poverty = row
            county_info = name.split(", ")
            county_name = county_info[0]
            state_abbr = county_info[1]

            # Match county with corresponding node in Neo4j
            result = session.run("""
                MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
                RETURN c
                """, state_abbr=state_abbr, county_name=county_name)

            for record in result:
                county_node = record["c"]

                # Update or create poverty level properties
                county_node.update({
                    "IncomeBelowPoverty": below_poverty,
                    "IncomeAtOrAbovePoverty": at_or_above_poverty
                })
                print(f"Poverty data populated for County {county_name}.")


def populate_housing_data(driver, df_S2502):
    with driver.session(database="neo4j") as session:
        for index, row in df_S2502.iterrows():
            geo_id, name, *housing_data, above_35 = row

            # Extract county name and state abbreviation
            county_info = name.split(", ")
            county_name = county_info[0]
            state_abbr = county_info[1]

            # Match county with corresponding node in Neo4j
            result = session.run("""
                MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
                RETURN c
                """, state_abbr=state_abbr, county_name=county_name)

            for record in result:
                county_node = record["c"]

                # Update or create housing data properties
                housing_data_dict = {column: value for column, value in zip(df_S2502.columns[2:], housing_data)}
                housing_data_dict.update({"Above35YearsOld": above_35})

                county_node.update(housing_data_dict)
                print(f"Housing data (S2502) populated for County {county_name}.")


def populate_housing_data_S2503(driver, df_S2503):
    with driver.session(database="neo4j") as session:
        for index, row in df_S2503.iterrows():
            geo_id, name, *housing_data = row

            # Extract county name and state abbreviation
            county_info = name.split(", ")
            county_name = county_info[0]
            state_abbr = county_info[1]

            # Match county with corresponding node in Neo4j
            result = session.run("""
                MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
                RETURN c
                """, state_abbr=state_abbr, county_name=county_name)

            for record in result:
                county_node = record["c"]

                # Update or create housing data properties
                housing_data_dict = {column: value for column, value in zip(df_S2503.columns[2:], housing_data)}
                county_node.update(housing_data_dict)
                print(f"Housing data (S2503) populated for County {county_name}.")



main()

BufferError: Existing exports of data: object cannot be re-sized

In [109]:
def populate_poverty_data(driver, B17017_data):
    with driver.session(database="neo4j") as session:
        for row in B17017_data[1:]:  # Skip header row
            geo_id, name, below_poverty, at_or_above_poverty = row
            county_info = name.split(", ")
            county_name = county_info[0]
            state_abbr = county_info[1]

            # Match county with corresponding node in Neo4j
            result = session.run("""
                MATCH (s:State)-[:HAS_COUNTY]->(c:County {name: $county_name})
                WHERE s.abbr = $state_abbr
                SET c.IncomeBelowPoverty = $below_poverty, c.IncomeAtOrAbovePoverty = $at_or_above_poverty
                """, county_name=county_name, state_abbr=state_abbr, below_poverty=below_poverty, at_or_above_poverty=at_or_above_poverty)



def populate_housing_data(driver, df):
    with driver.session(database="neo4j") as session:
        for index, row in df.iterrows():
            geo_id, name, *housing_data = row

            # Extract county name
            county_name = name.split(", ")[0]

            # Check if housing data is missing
            if len(housing_data) != 16:
                print(f"Missing housing data for {county_name}.")
                continue

            # Construct the Cypher query to update housing data for the county node
            query = """
                MATCH (c:County {name: $county_name})
                SET c.`Total Occupied Housing Units` = $total_units,
                    c.`Two or More Races` = $two_or_more_races,
                    c.`Hispanic or Latino Origin` = $hispanic_or_latino,
                    c.`White Alone, Not Hispanic or Latino` = $white_alone_not_hispanic,
                    c.`Under 35 Years Old` = $under_35,
                    c.`35 to 44 Years Old` = $age_35_to_44,
                    c.`45 to 54 Years Old` = $age_45_to_54,
                    c.`55 to 64 Years Old` = $age_55_to_64,
                    c.`65 to 74 Years Old` = $age_65_to_74,
                    c.`75 to 84 Years Old` = $age_75_to_84,
                    c.`85 Years Old and Over` = $age_85_and_over,
                    c.`Median Value` = $median_value,
                    c.`Median Gross Rent` = $median_gross_rent,
                    c.`Median Selected Monthly Owner Costs with Mortgage` = $median_selected_monthly_owner_costs_with_mortgage,
                    c.`Median Selected Monthly Owner Costs without Mortgage` = $median_selected_monthly_owner_costs_without_mortgage,
                    c.`Median Rent` = $median_rent
            """
            # Execute the query with housing data
            session.run(query, {
                "county_name": county_name,
                "total_units": housing_data[0],
                "two_or_more_races": housing_data[1],
                "hispanic_or_latino": housing_data[2],
                "white_alone_not_hispanic": housing_data[3],
                "under_35": housing_data[4],
                "age_35_to_44": housing_data[5],
                "age_45_to_54": housing_data[6],
                "age_55_to_64": housing_data[7],
                "age_65_to_74": housing_data[8],
                "age_75_to_84": housing_data[9],
                "age_85_and_over": housing_data[10],
                "median_value": housing_data[11],
                "median_gross_rent": housing_data[12],
                "median_selected_monthly_owner_costs_with_mortgage": housing_data[13],
                "median_selected_monthly_owner_costs_without_mortgage": housing_data[14],
                "median_rent": housing_data[15]
            })



URI = "neo4j+s://5dd34660.databases.neo4j.io"
AUTH = ("neo4j", "a1yUVTX6sa3eGz7MSVAqED0zzIB_VUq4YAsSPlVB-JU")
# Adjust the main function accordingly
def main():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            # Populate Neo4j with poverty data
            populate_poverty_data(driver, B17017_data)

            # Populate Neo4j with housing data from S2502 dataset
            populate_housing_data(driver, df_S2502)

            # Populate Neo4j with housing data from S2503 dataset
            populate_housing_data(driver, df_S2503)


main()


Missing housing data for Baldwin County.
Missing housing data for Calhoun County.
Missing housing data for Cullman County.
Missing housing data for DeKalb County.
Missing housing data for Elmore County.
Missing housing data for Etowah County.
Missing housing data for Houston County.
Missing housing data for Jefferson County.
Missing housing data for Lauderdale County.
Missing housing data for Lee County.
Missing housing data for Limestone County.
Missing housing data for Madison County.
Missing housing data for Marshall County.
Missing housing data for Mobile County.
Missing housing data for Montgomery County.
Missing housing data for Morgan County.
Missing housing data for St. Clair County.
Missing housing data for Shelby County.
Missing housing data for Talladega County.
Missing housing data for Tuscaloosa County.
Missing housing data for Anchorage Municipality.
Missing housing data for Fairbanks North Star Borough.
Missing housing data for Matanuska-Susitna Borough.
Missing housing 

In [110]:
from neo4j import GraphDatabase

# Define the Neo4j connection details
URI = "neo4j+s://5dd34660.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "a1yUVTX6sa3eGz7MSVAqED0zzIB_VUq4YAsSPlVB-JU"

# Function to execute the Cypher query and return the count
def execute_query_and_return_count(query):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            result = session.run(query)
            count = result.single()["count"]
            return count

# Query to check if poverty data was populated
poverty_query = """
MATCH (c:County)
WHERE c.IncomeBelowPoverty IS NOT NULL AND c.IncomeAtOrAbovePoverty IS NOT NULL
RETURN COUNT(c) AS count
"""

# Query to check if housing data from S2502 dataset was populated
housing_s2502_query = """
MATCH (c:County)
WHERE c.`Total Occupied Housing Units` IS NOT NULL
RETURN COUNT(c) AS count
"""

# Query to check if housing data from S2503 dataset was populated
housing_s2503_query = """
MATCH (c:County)
WHERE c.`Less than $20,000,Less than 20 percent` IS NOT NULL
RETURN COUNT(c) AS count
"""

# Execute queries and print results
print("Poverty data populated for", execute_query_and_return_count(poverty_query), "counties.")
print("Housing data (S2502) populated for", execute_query_and_return_count(housing_s2502_query), "counties.")
print("Housing data (S2503) populated for", execute_query_and_return_count(housing_s2503_query), "counties.")


Poverty data populated for 0 counties.
Housing data (S2502) populated for 628 counties.
Housing data (S2503) populated for 0 counties.
